 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Set-up-data" data-toc-modified-id="Set-up-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up data</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#RNNs-with-only-temperature-data" data-toc-modified-id="RNNs-with-only-temperature-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>RNNs with only temperature data</a></span><ul class="toc-item"><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#RNN-predicting-only-the-last-target" data-toc-modified-id="RNN-predicting-only-the-last-target-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>RNN predicting only the last target</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Sequence-RNN" data-toc-modified-id="Sequence-RNN-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Sequence RNN</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Longer-sequence" data-toc-modified-id="Longer-sequence-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Longer sequence</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Add-dropout" data-toc-modified-id="Add-dropout-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Add dropout</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Reference-experiment-with-longer-training-set." data-toc-modified-id="Reference-experiment-with-longer-training-set.-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Reference experiment with longer training set.</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Sequence-model-with-longer-training-set" data-toc-modified-id="Sequence-model-with-longer-training-set-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Sequence model with longer training set</a></span></li></ul></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Get-additional-variables" data-toc-modified-id="Get-additional-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get additional variables</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Combining-with-embeddings" data-toc-modified-id="Combining-with-embeddings-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combining with embeddings</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Embedding-with-only-temperature" data-toc-modified-id="Embedding-with-only-temperature-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Embedding with only temperature</a></span></li></ul></div>

# Recurrent neural networks

In this notebook we will try out RNNs for our post-processing. The idea here is that there might be some extra information in looking at data from previous time steps.

RNNs take quite a long time to train, so I am using a GPU here.

In [156]:
# Imports
from importlib import reload
import utils; reload(utils)
from utils import *
import crps_loss; reload(crps_loss)
from crps_loss import crps_cost_function, crps_cost_function_seq
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout, \
    SimpleRNN, LSTM, TimeDistributed, GRU, Dropout, Masking
from keras.layers.merge import Concatenate
from keras.models import Model, Sequential
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

Anaconda environment: py36_gpu
Linux 4.4.0-96-generic


In [2]:
# Use this if you want to limit the GPU RAM usage
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

In [3]:
# Basic setup
# DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Set up data

This is now also done inside the `get_train_test_sets` function. `seq_len` is the number of timesteps (including the one to predict). We will start out with a moderate length of 5 days, training for 2015, predicting for 2016.

In [4]:
seq_len=5

In [73]:
train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates, 
                                          seq_len=seq_len, fill_value=-999.)

train set contains 365 days
test set contains 366 days


In [74]:
train_set.features.shape, train_set.targets.shape

((180849, 5, 2), (180849, 5, 1))

The arrays have dimensions [sample, time step, feature]

## RNNs with only temperature data

As a comparison. Our simple networks got a train/test loss of around 1.07/1.01.

I am using a Gated Recurrent Unit (GRU) as my recurrent layer. LSTM is probably the more common one, but GRU is slightly cheaper and for our simple applications provides similar results. 

### RNN predicting only the last target

In [75]:
batch_size = 1024
hidden_nodes = 100   # Number of hidden nodes inside RNN cell

In [76]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes)(inp)
x = Dense(2, activation='linear')(x)
rnn_model = Model(inputs=inp, outputs=x)

In [77]:
rnn_model.compile(optimizer=Adam(0.01), loss=crps_cost_function)

In [78]:
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 5, 2)              0         
_________________________________________________________________
gru_3 (GRU)                  (None, 100)               30900     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 31,102
Trainable params: 31,102
Non-trainable params: 0
_________________________________________________________________


In [79]:
rnn_model.fit(train_set.features, train_set.targets[:,-1], epochs=10, batch_size=batch_size,
              validation_data=(test_set.features, test_set.targets[:,-1]))

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 4s - loss: 1.8537 - val_loss: 1.0207
Epoch 2/10
180849/180849 [==============================] - 3s - loss: 1.0508 - val_loss: 1.0188
Epoch 3/10
180849/180849 [==============================] - 3s - loss: 1.0394 - val_loss: 1.0122
Epoch 4/10
180849/180849 [==============================] - 3s - loss: 1.0360 - val_loss: 1.0078
Epoch 5/10
180849/180849 [==============================] - 3s - loss: 1.0333 - val_loss: 1.0066
Epoch 6/10
180849/180849 [==============================] - 3s - loss: 1.0287 - val_loss: 1.0079
Epoch 7/10
180849/180849 [==============================] - 3s - loss: 1.0280 - val_loss: 1.0193
Epoch 8/10
180849/180849 [==============================] - 3s - loss: 1.0236 - val_loss: 1.0325
Epoch 9/10
180849/180849 [==============================] - 3s - loss: 1.0190 - val_loss: 1.0185
Epoch 10/10
180849/180849 [==============================] - 3s - loss: 1.0

So we get a better train score and a worse validation score. This indicates overfitting. 

### Sequence RNN

In [80]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes, return_sequences=True)(inp)
x = TimeDistributed(Dense(2, activation='linear'))(x)
seq_rnn_model = Model(inputs=inp, outputs=x)

In [81]:
seq_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5, 2)              0         
_________________________________________________________________
gru_4 (GRU)                  (None, 5, 100)            30900     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 2)              202       
Total params: 31,102
Trainable params: 31,102
Non-trainable params: 0
_________________________________________________________________


In [82]:
seq_rnn_model.compile(optimizer=Adam(0.01), loss=crps_cost_function_seq, 
                      sample_weight_mode="temporal")

In [148]:
def train_and_valid(model, train_set, test_set, epochs, batch_size, verbose=0, emb=False):
    """Write our own function to train and validate, 
    because the keras fit function cannot handle sample weights for training
    and validation at the same time.
    """
    train_inp = [train_set.features, train_set.cont_ids] if emb else train_set.features
    test_inp = [test_set.features, test_set.cont_ids] if emb else test_set.features
    for i in range(epochs):
        print('Epoch:', i+1)
        t1 = timeit.default_timer()
        
        h = model.fit(train_inp, train_set.targets, epochs=1, batch_size=batch_size, 
                      sample_weight=train_set.sample_weights, verbose=verbose)
        t2 = timeit.default_timer()
        print('Train loss: %.4f - Valid loss: %.4f - Time: %.1fs' % (h.history['loss'][0],  
                    model.evaluate(test_inp, test_set.targets, batch_size=10000, 
                       sample_weight=test_set.sample_weights, verbose=verbose), 
                    t2 - t1))

In [88]:
train_and_valid(seq_rnn_model, train_set, test_set, 10, batch_size)

Epoch: 1
Train loss: 1.0207 - Valid loss: 1.0338
Epoch: 2
Train loss: 1.0179 - Valid loss: 1.0420
Epoch: 3
Train loss: 1.0173 - Valid loss: 1.0316
Epoch: 4
Train loss: 1.0161 - Valid loss: 1.0381
Epoch: 5
Train loss: 1.0135 - Valid loss: 1.0471
Epoch: 6
Train loss: 1.0123 - Valid loss: 1.0401
Epoch: 7
Train loss: 1.0117 - Valid loss: 1.0343
Epoch: 8
Train loss: 1.0089 - Valid loss: 1.0547
Epoch: 9
Train loss: 1.0078 - Valid loss: 1.0515
Epoch: 10
Train loss: 1.0064 - Valid loss: 1.0479


Same as with the first RNN above we seem to overfit to the dataset, but maybe not as strongly. Let's now try a more complex model with a longer sequence length.

### Longer sequence

In [112]:
seq_len = 20
train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates, 
                                          seq_len=seq_len, fill_value=-999.)

train set contains 365 days
test set contains 366 days


In [90]:
hidden_nodes = 200

In [91]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes, return_sequences=True)(inp)
x = TimeDistributed(Dense(2, activation='linear'))(x)
seq_rnn_model = Model(inputs=inp, outputs=x)

In [92]:
seq_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 20, 2)             0         
_________________________________________________________________
gru_5 (GRU)                  (None, 20, 200)           121800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 2)             402       
Total params: 122,202
Trainable params: 122,202
Non-trainable params: 0
_________________________________________________________________


In [93]:
seq_rnn_model.compile(optimizer=Adam(0.01), loss=crps_cost_function_seq, 
                      sample_weight_mode="temporal")

In [94]:
train_and_valid(seq_rnn_model, train_set, test_set, 10, batch_size)

Epoch: 1
Train loss: 1.7379 - Valid loss: 1.0252
Epoch: 2
Train loss: 1.0267 - Valid loss: 1.0240
Epoch: 3
Train loss: 1.0123 - Valid loss: 1.0222
Epoch: 4
Train loss: 0.9968 - Valid loss: 1.0459
Epoch: 5
Train loss: 0.9759 - Valid loss: 1.0582
Epoch: 6
Train loss: 0.9524 - Valid loss: 1.0647
Epoch: 7
Train loss: 0.9270 - Valid loss: 1.0764
Epoch: 8
Train loss: 0.9082 - Valid loss: 1.0816
Epoch: 9
Train loss: 0.8912 - Valid loss: 1.1043
Epoch: 10
Train loss: 0.8779 - Valid loss: 1.0920


So again we are overfitting, but maybe there is something to be learned. Let's first add some regularization and then try a longer training set.

### Add dropout

In [95]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes, return_sequences=True, recurrent_dropout=0.5)(inp)
x = TimeDistributed(Dense(2, activation='linear'))(x)
seq_rnn_model = Model(inputs=inp, outputs=x)
seq_rnn_model.compile(optimizer=Adam(0.001), loss=crps_cost_function_seq, 
                      sample_weight_mode="temporal")

In [97]:
train_and_valid(seq_rnn_model, train_set, test_set, 10, batch_size)

Epoch: 1
Train loss: 1.0384 - Valid loss: 1.0048
Epoch: 2
Train loss: 1.0361 - Valid loss: 1.0030
Epoch: 3
Train loss: 1.0342 - Valid loss: 1.0019
Epoch: 4
Train loss: 1.0332 - Valid loss: 1.0035
Epoch: 5
Train loss: 1.0319 - Valid loss: 1.0027
Epoch: 6
Train loss: 1.0309 - Valid loss: 1.0033
Epoch: 7
Train loss: 1.0301 - Valid loss: 1.0073
Epoch: 8
Train loss: 1.0290 - Valid loss: 1.0035
Epoch: 9
Train loss: 1.0284 - Valid loss: 1.0086
Epoch: 10
Train loss: 1.0278 - Valid loss: 1.0054


So with drop out we get slightly better validation results, but we are still starting to overfit. I think there is a lot of parameter tuning that would be possible with the complexity of the network and so forth. 

### Reference experiment with longer training set.

In [98]:
train_dates_long = ['2008-01-01', '2016-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates_long, test_dates)

train set contains 2922 days
test set contains 366 days


In [99]:
# Copied from fc_network notebook
def build_fc_model():
    inp = Input(shape=(2,))
    x = Dense(2, activation='linear')(inp)
    return Model(inputs=inp, outputs=x)

In [100]:
fc_model = build_fc_model()
fc_model.compile(optimizer=Adam(0.1), loss=crps_cost_function)

In [101]:
fc_model.fit(train_set.features, train_set.targets, epochs=10, batch_size=1024,
             validation_data=[test_set.features, test_set.targets])

Train on 1456977 samples, validate on 182218 samples
Epoch 1/10
1456977/1456977 [==============================] - 6s - loss: 1.3737 - val_loss: 1.0100
Epoch 2/10
1456977/1456977 [==============================] - 5s - loss: 1.0626 - val_loss: 1.0109
Epoch 3/10
1456977/1456977 [==============================] - 5s - loss: 1.0628 - val_loss: 1.0079
Epoch 4/10
1456977/1456977 [==============================] - 5s - loss: 1.0630 - val_loss: 1.0126
Epoch 5/10
1456977/1456977 [==============================] - 5s - loss: 1.0630 - val_loss: 1.0109
Epoch 6/10
1456977/1456977 [==============================] - 5s - loss: 1.0630 - val_loss: 1.0074
Epoch 7/10
1456977/1456977 [==============================] - 5s - loss: 1.0629 - val_loss: 1.0119
Epoch 8/10
1456977/1456977 [==============================] - 5s - loss: 1.0630 - val_loss: 1.0130
Epoch 9/10
1456977/1456977 [==============================] - 5s - loss: 1.0630 - val_loss: 1.0082
Epoch 10/10
1456977/1456977 [===========================

Maybe a small improvement. Now let's test our sequence model with a longer training period.

### Sequence model with longer training set

In [102]:
seq_len = 20
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates_long, test_dates, 
                                          seq_len=seq_len, fill_value=-999.)

train set contains 2922 days
test set contains 366 days


In [107]:
def build_seq_rnn(hidden_nodes, n_features, dropout=0, lr=0.01):
    inp = Input(shape=(seq_len, n_features, )) # time step, feature
    x = GRU(hidden_nodes, return_sequences=True, recurrent_dropout=dropout)(inp)
    x = TimeDistributed(Dense(2, activation='linear'))(x)
    seq_rnn_model = Model(inputs=inp, outputs=x)
    seq_rnn_model.compile(optimizer=Adam(lr), loss=crps_cost_function_seq, 
                          sample_weight_mode="temporal")
    return seq_rnn_model

In [104]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes, return_sequences=True, recurrent_dropout=0.5)(inp)
x = TimeDistributed(Dense(2, activation='linear'))(x)
seq_rnn_model = Model(inputs=inp, outputs=x)
seq_rnn_model.compile(optimizer=Adam(0.001), loss=crps_cost_function_seq, 
                      sample_weight_mode="temporal")

In [106]:
# This takes several minutes on the GPU
# Epoch counter: 7
train_and_valid(seq_rnn_model, train_set, test_set, 2, batch_size)

Epoch: 1
Train loss: 1.0349 - Valid loss: 0.9941
Epoch: 2
Train loss: 1.0333 - Valid loss: 0.9959


## Get additional variables

In [109]:
from collections import OrderedDict
aux_dict = OrderedDict()
aux_dict['data_aux_geo_interpolated.nc'] = ['orog', 
                                            'station_alt', 
                                            'station_lat', 
                                            'station_lon']
aux_dict['data_aux_pl500_interpolated_00UTC.nc'] = ['u_pl500_fc',
                                                    'v_pl500_fc',
                                                    'gh_pl500_fc']
aux_dict['data_aux_pl850_interpolated_00UTC.nc'] = ['u_pl850_fc',
                                                    'v_pl850_fc',
                                                    'q_pl850_fc']
aux_dict['data_aux_surface_interpolated_00UTC.nc'] = ['cape_fc',
                                                      'sp_fc',
                                                      'tcc_fc']

In [157]:
# Start with just one training year
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates, 
                                          seq_len=seq_len, fill_value=-999., aux_dict=aux_dict)

train set contains 365 days
test set contains 366 days


In [158]:
train_set.cont_ids.shape

(180849, 20)

In [111]:
n_features = train_set.features.shape[-1]
n_features

24

In [114]:
seq_rnn_model = build_seq_rnn(hidden_nodes, n_features, dropout=0.5, lr=0.01)

In [121]:
# Epoch counter: 8
train_and_valid(seq_rnn_model, train_set, test_set, 2, batch_size, verbose=1)

Epoch: 1
Epoch 1/1
182218/182218 [==============================] - 6s     
Train loss: 0.8329 - Valid loss: 0.9341 - Time: 18.1s
Epoch: 2
Epoch 1/1
182218/182218 [==============================] - 6s     
Train loss: 0.8201 - Valid loss: 0.9315 - Time: 18.1s


In [131]:
seq_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 20, 24)            0         
_________________________________________________________________
gru_9 (GRU)                  (None, 20, 200)           135000    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 2)             402       
Total params: 135,402
Trainable params: 135,402
Non-trainable params: 0
_________________________________________________________________


## Combining with embeddings

In [187]:
def build_seq_rnn_with_embeddings(seq_len, hidden_nodes, n_features, emb_size, max_id, 
                                  recurrent_dropout=0, dropout=0, lr=0.01):
    features_inp = Input(shape=(seq_len, n_features, )) # time step, feature
    id_in = Input(shape=(seq_len,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    x = GRU(hidden_nodes, return_sequences=True, recurrent_dropout=recurrent_dropout)(features_inp)
    x = Concatenate()([x, emb])
    x = Dropout(dropout)(x)
    x = TimeDistributed(Dense(2, activation='linear'))(x)
    model = Model(inputs=[features_inp, id_in], outputs=x)
    model.compile(optimizer=Adam(lr), loss=crps_cost_function_seq, 
                          sample_weight_mode="temporal")
    return model

In [163]:
emb_size = 5
max_id = int(np.max([train_set.cont_ids.max(), test_set.cont_ids.max()]))
hidden_nodes, max_id

(200, 536)

In [164]:
emb_rnn = build_seq_rnn_with_embeddings(hidden_nodes, n_features, emb_size, max_id, 0.5)

In [165]:
emb_rnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_24 (InputLayer)            (None, 20, 24)        0                                            
____________________________________________________________________________________________________
input_25 (InputLayer)            (None, 20)            0                                            
____________________________________________________________________________________________________
gru_16 (GRU)                     (None, 20, 200)       135000      input_24[0][0]                   
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 20, 5)         2685        input_25[0][0]                   
___________________________________________________________________________________________

In [167]:
train_and_valid(emb_rnn, train_set, test_set, 5, batch_size, emb=True)

Epoch: 1
Train loss: 0.9147 - Valid loss: 0.8970 - Time: 19.6s
Epoch: 2
Train loss: 0.9016 - Valid loss: 0.8984 - Time: 19.7s
Epoch: 3
Train loss: 0.8923 - Valid loss: 0.9004 - Time: 19.6s
Epoch: 4
Train loss: 0.8822 - Valid loss: 0.9023 - Time: 19.6s
Epoch: 5
Train loss: 0.8737 - Valid loss: 0.8983 - Time: 19.6s


## Embedding with only temperature

In [182]:
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates, 
                                          seq_len=2, fill_value=-999.)

train set contains 365 days
test set contains 366 days


In [191]:
emb_rnn = build_seq_rnn_with_embeddings(2, 10, 2, emb_size, max_id, recurrent_dropout=0)

In [192]:
emb_rnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_36 (InputLayer)            (None, 2, 2)          0                                            
____________________________________________________________________________________________________
input_37 (InputLayer)            (None, 2)             0                                            
____________________________________________________________________________________________________
gru_22 (GRU)                     (None, 2, 10)         390         input_36[0][0]                   
____________________________________________________________________________________________________
embedding_14 (Embedding)         (None, 2, 5)          2685        input_37[0][0]                   
___________________________________________________________________________________________

In [198]:
train_and_valid(emb_rnn, train_set, test_set, 5, batch_size, emb=True)

Epoch: 1
Train loss: 0.9538 - Valid loss: 0.9194 - Time: 1.2s
Epoch: 2
Train loss: 0.9534 - Valid loss: 0.9233 - Time: 1.1s
Epoch: 3
Train loss: 0.9541 - Valid loss: 0.9263 - Time: 1.1s
Epoch: 4
Train loss: 0.9538 - Valid loss: 0.9229 - Time: 1.2s
Epoch: 5
Train loss: 0.9536 - Valid loss: 0.9233 - Time: 1.1s


In [200]:
train_dates_long = ['2008-01-01', '2016-01-01']
train_set_long, test_set = get_train_test_sets(DATA_DIR, train_dates_long, test_dates, 
                                          seq_len=seq_len, fill_value=-999.)

train set contains 2922 days
test set contains 366 days


In [204]:
emb_rnn = build_seq_rnn_with_embeddings(seq_len, 30, 2, emb_size, max_id, recurrent_dropout=0.3)

In [205]:
train_and_valid(emb_rnn, train_set_long, test_set, 15, batch_size, emb=True)

Epoch: 1
Train loss: 1.2161 - Valid loss: 0.9024 - Time: 40.9s
Epoch: 2
Train loss: 0.9281 - Valid loss: 0.8962 - Time: 38.3s
Epoch: 3
Train loss: 0.9255 - Valid loss: 0.8949 - Time: 38.3s
Epoch: 4
Train loss: 0.9239 - Valid loss: 0.8965 - Time: 38.3s
Epoch: 5
Train loss: 0.9227 - Valid loss: 0.8954 - Time: 38.3s
Epoch: 6
Train loss: 0.9216 - Valid loss: 0.9010 - Time: 38.3s
Epoch: 7
Train loss: 0.9210 - Valid loss: 0.9060 - Time: 38.3s
Epoch: 8
Train loss: 0.9205 - Valid loss: 0.8914 - Time: 38.2s
Epoch: 9
Train loss: 0.9198 - Valid loss: 0.8937 - Time: 38.3s
Epoch: 10
Train loss: 0.9193 - Valid loss: 0.8931 - Time: 38.2s
Epoch: 11
Train loss: 0.9191 - Valid loss: 0.8955 - Time: 38.2s
Epoch: 12
Train loss: 0.9188 - Valid loss: 0.9047 - Time: 38.2s
Epoch: 13
Train loss: 0.9186 - Valid loss: 0.8932 - Time: 38.2s
Epoch: 14
Train loss: 0.9184 - Valid loss: 0.8945 - Time: 38.2s
Epoch: 15
Train loss: 0.9183 - Valid loss: 0.8993 - Time: 38.2s
